<a href="https://colab.research.google.com/github/JimMcDonough/Capstone/blob/main/Roberta_Full_training_and_testing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Sun Nov 20 19:46:52 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  A100-SXM4-40GB      Off  | 00000000:00:04.0 Off |                    0 |
| N/A   31C    P0    43W / 400W |      0MiB / 40536MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 89.6 gigabytes of available RAM

You are using a high-RAM runtime!


In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 5.1 MB/s 
     |████████████████████████████████| 182 kB 58.9 MB/s 
     |████████████████████████████████| 7.6 MB 82.8 MB/s 


In [ ]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 451 kB 29.9 MB/s 
     |████████████████████████████████| 212 kB 91.9 MB/s 
     |████████████████████████████████| 115 kB 79.8 MB/s 
     |████████████████████████████████| 127 kB 83.9 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3


In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
from datasets import load_dataset, Value, Sequence, Features

data_files={"train": r"/content/drive/My Drive/sample_data/full_train.json",
           "validation": r"/content/drive/My Drive/sample_data/full_val.json",
           "test": r"/content/drive/My Drive/sample_data/full_test.json"}

In [ ]:
features = Features({'answers': [{'text': Value(dtype='string'), 'answer_start': Value(dtype='int32')}],
'context': Value(dtype='string'),
'id': Value(dtype='string'),
'question': Value(dtype='string'),
'title': Value(dtype='string')})

In [ ]:
dataset = load_dataset("json", data_files=data_files, features=features, cache_dir= r"/content/drive/My Drive/sample_data/")

  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("roberta-base")

Downloading:   0%|          | 0.00/481 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:
tokenizer.is_fast

True

In [ ]:
from transformers import DefaultDataCollator
from transformers import TFAutoModelForQuestionAnswering
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.optimizers.schedules import PolynomialDecay
import tensorflow as tf
from tensorflow.keras.optimizers import Adam

data_collator = DefaultDataCollator(return_tensors="tf")


In [ ]:
results_list = []
max_length = 384
stride = 128
batch_size = 16
num_epochs = 3
learning_rate = 0.000064

In [ ]:
def preprocess_function(examples):
            questions = [q.strip() for q in examples["question"]]
            inputs = tokenizer(
                questions,
                examples["context"],
                max_length=max_length,
                truncation="only_second",  
                stride = stride,   #only truncates the context nothing else
                return_overflowing_tokens=True,
                return_offsets_mapping=True,  # map the start and end positions of the answer to the original context 
                padding="max_length",
            )
            
            offset_mapping = inputs.pop("offset_mapping")
            #print(offset_mapping)
            sample_map = inputs.pop("overflow_to_sample_mapping")
            answers = examples["answers"]
            start_positions = []
            end_positions = []
            #print(answers)
            for i, offset in enumerate(offset_mapping):
                sample_idx = sample_map[i]
                answer = answers[sample_idx]
                #start_char = answer["answer_start"][0]
                #end_char = answer["answer_start"][0] + len(answer["text"][0])  #start char + length giver chars with answer
                start_char = answer[0]["answer_start"]
                end_char = answer[0]["answer_start"] + len(answer[0]["text"])  
                sequence_ids = inputs.sequence_ids(i)  #no clue what this does
                #print(answer, end_char, sequence_ids)
                # Find the start and end of the context        ; not sure how this section works
                idx = 0
                while sequence_ids[idx] != 1:
                    idx += 1
                context_start = idx
                while sequence_ids[idx] == 1:
                    idx += 1
                context_end = idx - 1
            
                # If the answer is not fully inside the context, label it (0, 0)
                if offset[context_start][0] > start_char or offset[context_end][1] < end_char: 
                    start_positions.append(0)
                    end_positions.append(0)
                else:
                    # Otherwise it's the start and end token positions
                    idx = context_start
                    while idx <= context_end and offset[idx][0] <= start_char:
                        idx += 1
                    start_positions.append(idx - 1)
                    
                    idx = context_end
                    while idx >= context_start and offset[idx][1] >= end_char:
                        idx -= 1
                    end_positions.append(idx + 1)
                    
            inputs["start_positions"] = start_positions
            inputs["end_positions"] = end_positions
            return inputs 

In [ ]:
tokenized_squad = dataset.map(preprocess_function, batched=True, remove_columns=dataset["train"].column_names)

          

  0%|          | 0/71 [00:00<?, ?ba/s]

  0%|          | 0/8 [00:00<?, ?ba/s]

  0%|          | 0/9 [00:00<?, ?ba/s]

In [ ]:
model = TFAutoModelForQuestionAnswering.from_pretrained("roberta-base")

Downloading:   0%|          | 0.00/657M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFRobertaForQuestionAnswering.

Some layers of TFRobertaForQuestionAnswering were not initialized from the model checkpoint at roberta-base and are newly initialized: ['qa_outputs']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
tf_train_set = model.prepare_tf_dataset(
    tokenized_squad["train"],
    shuffle=True,
    batch_size=batch_size,
    collate_fn=data_collator,
)
          
tf_validation_set = model.prepare_tf_dataset(
    tokenized_squad["validation"],
    shuffle=False,
    batch_size=batch_size,
    collate_fn=data_collator,
)

num_train_steps = len(tf_train_set) * num_epochs
lr_scheduler = PolynomialDecay(
    initial_learning_rate=learning_rate, end_learning_rate=0.0, decay_steps=num_train_steps
)

opt = Adam(learning_rate=lr_scheduler)

loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer=opt, loss=loss, metrics=["accuracy"])

In [ ]:
history = model.fit(tf_train_set, validation_data=tf_validation_set, epochs=num_epochs)

Epoch 1/3
4461/4461 [==============================] - 758s 167ms/step - loss: 2.5272 - end_logits_loss: 1.2013 - start_logits_loss: 1.3260 - end_logits_accuracy: 0.6838 - start_logits_accuracy: 0.6430 - val_loss: 1.8904 - val_end_logits_loss: 0.8699 - val_start_logits_loss: 1.0205 - val_end_logits_accuracy: 0.7477 - val_start_logits_accuracy: 0.6883
Epoch 2/3
4461/4461 [==============================] - 740s 166ms/step - loss: 1.5864 - end_logits_loss: 0.7446 - start_logits_loss: 0.8418 - end_logits_accuracy: 0.7831 - start_logits_accuracy: 0.7425 - val_loss: 1.7421 - val_end_logits_loss: 0.8011 - val_start_logits_loss: 0.9411 - val_end_logits_accuracy: 0.7641 - val_start_logits_accuracy: 0.7018
Epoch 3/3
4461/4461 [==============================] - 740s 166ms/step - loss: 1.0921 - end_logits_loss: 0.5022 - start_logits_loss: 0.5898 - end_logits_accuracy: 0.8410 - start_logits_accuracy: 0.8010 - val_loss: 1.7889 - val_end_logits_loss: 0.8198 - val_start_logits_loss: 0.9691 - val_end_l

In [ ]:
results_list = []
max_length = 384
stride = 50
batch_size = 16
num_epochs = 3
learning_rate = 0.000064

In [ ]:
tokenized_squad = dataset.map(preprocess_function, batched=True, remove_columns=dataset["train"].column_names)

  0%|          | 0/71 [00:00<?, ?ba/s]

  0%|          | 0/8 [00:00<?, ?ba/s]

  0%|          | 0/9 [00:00<?, ?ba/s]

In [ ]:
model = TFAutoModelForQuestionAnswering.from_pretrained("roberta-base")

All model checkpoint layers were used when initializing TFRobertaForQuestionAnswering.

Some layers of TFRobertaForQuestionAnswering were not initialized from the model checkpoint at roberta-base and are newly initialized: ['qa_outputs']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
tf_train_set = model.prepare_tf_dataset(
    tokenized_squad["train"],
    shuffle=True,
    batch_size=batch_size,
    collate_fn=data_collator,
)
          
tf_validation_set = model.prepare_tf_dataset(
    tokenized_squad["validation"],
    shuffle=False,
    batch_size=batch_size,
    collate_fn=data_collator,
)

num_train_steps = len(tf_train_set) * num_epochs
lr_scheduler = PolynomialDecay(
    initial_learning_rate=learning_rate, end_learning_rate=0.0, decay_steps=num_train_steps
)

opt = Adam(learning_rate=lr_scheduler)

loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
model.compile(optimizer=opt, loss=loss, metrics=["accuracy"])

In [ ]:
history = model.fit(tf_train_set, validation_data=tf_validation_set, epochs=num_epochs)

Epoch 1/3
4460/4460 [==============================] - 755s 167ms/step - loss: 2.2658 - end_logits_loss: 1.0843 - start_logits_loss: 1.1815 - end_logits_accuracy: 0.7033 - start_logits_accuracy: 0.6643 - val_loss: 1.8171 - val_end_logits_loss: 0.8491 - val_start_logits_loss: 0.9681 - val_end_logits_accuracy: 0.7475 - val_start_logits_accuracy: 0.6947
Epoch 2/3
4460/4460 [==============================] - 742s 166ms/step - loss: 1.4466 - end_logits_loss: 0.6842 - start_logits_loss: 0.7624 - end_logits_accuracy: 0.7978 - start_logits_accuracy: 0.7584 - val_loss: 1.7940 - val_end_logits_loss: 0.8242 - val_start_logits_loss: 0.9698 - val_end_logits_accuracy: 0.7629 - val_start_logits_accuracy: 0.7075
Epoch 3/3
4460/4460 [==============================] - 742s 166ms/step - loss: 0.9604 - end_logits_loss: 0.4480 - start_logits_loss: 0.5124 - end_logits_accuracy: 0.8601 - start_logits_accuracy: 0.8227 - val_loss: 1.8699 - val_end_logits_loss: 0.8642 - val_start_logits_loss: 1.0057 - val_end_l

In [ ]:
#https://huggingface.co/docs/transformers/v4.23.1/en/serialization
model.save_pretrained(r"/content/drive/My Drive/sample_data/saved_model/")
tokenizer.save_pretrained(r"/content/drive/My Drive/sample_data/saved_model/")

('/content/drive/My Drive/sample_data/saved_model/tokenizer_config.json',
 '/content/drive/My Drive/sample_data/saved_model/special_tokens_map.json',
 '/content/drive/My Drive/sample_data/saved_model/vocab.json',
 '/content/drive/My Drive/sample_data/saved_model/merges.txt',
 '/content/drive/My Drive/sample_data/saved_model/added_tokens.json',
 '/content/drive/My Drive/sample_data/saved_model/tokenizer.json')

In [ ]:
from transformers import pipeline
question_answerer = pipeline(task="question-answering", model=model, tokenizer=tokenizer)

In [ ]:
exact_answers = []
contains_answer = []
wrong = []
scores = []
for text_row in dataset['test']:
    result = question_answerer(question = text_row['question'], context = text_row['context'])
    correct_start = text_row['answers'][0]['answer_start'] 
    correct_end = text_row['answers'][0]['answer_start'] + len(text_row['answers'][0]['text'])
    scores.append(result['score'])
   
    if (correct_start == result['start']) and (correct_end == result['end']):
        exact_answers.append(1)
        contains_answer.append(1)
        wrong.append(0)
   
    
    elif (correct_start != result['start']) or (correct_end != result['end']):
        
        if (correct_start >= result['start']) and (correct_end <= result['end']):
            exact_answers.append(0)
            contains_answer.append(1)
            wrong.append(0)
        else:
            exact_answers.append(0)
            contains_answer.append(0)
            wrong.append(1)
    else:
        pass

In [ ]:
Accuracy = sum(exact_answers)/len(dataset['test'])
Accuracy

0.6371102118543204

In [ ]:
containment_accuracy = sum(contains_answer)/len(dataset['test'])
containment_accuracy

0.7233991906688884